# Automating the interpretation of calcium assay data 

Annotations specific to this run through for assessment are written in **. They are not included in the usual code.

## Importing required packages

In [ ]:
# These are the packages we'll be using:

import pandas as pd
import matplotlib.pyplot as plt
import math

## Reading in data
I found that I could only import data from my github if i called it in the raw format, here i've done this automatically so that you can copy the URL directly from the github website.

In [ ]:
url = 'https://github.com/U1711/calcium_project/blob/main/Fucosidosis%20A%2013032018.csv' #<--- insert URL here

# * I have already included a test document at the specified URL*

print("The URL is:", url)


#Data doesn't seem to import unless it is in 'raw' format
raw_url = url.replace("blob", "raw")

print("\nOur converted raw URL is:", raw_url)


# Reading in our data
test_dataset = pd.read_csv(raw_url)

print("\nThis is the whole dataset below:\n\n", test_dataset)

The amount of columns here looks a bit messy, only some are of interest... 

## Tidying up test_dataset

The columns of interest are Time (sec) and the R[i] R1 columns.
The first R refers to the Region, in this case individual cells.
The second R1 is the ratio between W1 and W2, this is what we're interested in and as the ratio has already been calculated for us we no longer need W1 and W2.

### Deleting unwanted columns

In [ ]:
# Identifying and deleting R[i] W1 Avg columns
Ri_W1_Avg_columns = []

for i in range(1, 51):
    column_name = 'R' + str(i) + ' W1 Avg'
    if column_name in test_dataset.columns:
        Ri_W1_Avg_columns.append(column_name)

if len(Ri_W1_Avg_columns) > 0:
    test_dataset = test_dataset.drop(columns=Ri_W1_Avg_columns)

    
# Identifying and deleting R[i] W2 Avg columns
Ri_W2_Avg_columns = []

for i in range(1, 51):
    column_name = 'R' + str(i) + ' W2 Avg'
    if column_name in test_dataset.columns:
        Ri_W2_Avg_columns.append(column_name)

if len(Ri_W2_Avg_columns) > 0:
    test_dataset = test_dataset.drop(columns=Ri_W2_Avg_columns)

    
# Deleting columns where all the values are NaN
test_dataset = test_dataset.dropna(axis=1, how="all")


# Deleting columns that contain only 0 and NaN values
columns = test_dataset.columns[test_dataset.isin([0, pd.np.nan]).all() & test_dataset.notnull().any(axis=0)]

test_dataset = test_dataset.drop(columns=columns)


# Deleting unnamed columns
test_dataset = test_dataset.drop(columns=[col for col in test_dataset.columns if "Unnamed:" in col])


print(" This is the test dataset without: R[i] W1 or W2 Avg columns, NaN columns, blank columns or unnamed columns:\n\n", test_dataset)

### Naming our data

In [ ]:
# Assigning the 'Time (sec)' column the name 'time'

time = test_dataset["Time (sec)"]


# Assigning the R[i] R1 columns the name calcium_r[i]

for i in range(1, 51):
    try:
        exec(f'calcium_r{i} = test_dataset["R{i} R1"].values')
    except KeyError:

        continue


print("Our time dataset looks like this:\n\n", time, "\n\n\n\n And an example of our calcium_r[i] (9) dataset looks like this:\n\n", calcium_r9)

## Visualising all of our calcium_r[i] data against time

In [ ]:
# Setting 25 rows of 2 columns (enough for up to 50 calcium_r[i])
nrows = math.ceil(50 / 2)
ncols = 2

fig, axs = plt.subplots(nrows, ncols, figsize=(10, nrows*5)) # Creating figure/subplots

axs = axs.flatten() # Flattening the subplot array to make it easier to iterate over

# Looping through the variables to plot each one
for i, ax in enumerate(axs):
    calcium_r = "calcium_r" + str(i+1)
    if calcium_r in locals():
        ax.plot(time, locals()[calcium_r], 'b.')
        ax.set_title("Calcium {}/Time".format(i+1))
        ax.set_ylabel("Calcium {}".format(i+1))
        ax.set_xlabel("Time (sec)")

The data seems to consist of swift incline (the action of a drug) which comes to a peak followed by a small lag period and a swift decline. This decline then plateaus to a trough, and finally the data tails off and rises again.
I am interested in identifying the steep rate of decline after the lag phase but before the decline begins to plateau - this should be a fairly consistent measure of how quickly calcium levels return to normal.

## Identifying the rate at which calcium levels return to normal following the peak

This code was designed to call a range of datasets with up to 50 regions/columns.

In [ ]:
# Combining calcium_r[i] and time into a list of tuples
for i in range(1, 51):
    try:
        globals()['time_tuples' + str(i)] = list(zip(time, locals()['calcium_r' + str(i)]))
    except KeyError:
        pass


# Calculating the gradient between the tuples
for i in range(1, 51):
    try:
        gradients = []
        for j in range(len(globals()['time_tuples' + str(i)])-1):
            gradient = (globals()['time_tuples' + str(i)][j+1][1] - globals()['time_tuples' + str(i)][j][1]) / (globals()['time_tuples' + str(i)][j+1][0] - globals()['time_tuples' + str(i)][j][0])
            gradients.append(gradient)
        globals()['gradients' + str(i)] = gradients
    except KeyError:
        pass


# Defining a function to find the longest straight(ish) declining line (this should be the line of interest) based on a
    # tolerance of 'straightness'.
def calculate_average_gradient(times, values, tolerance):
  time_tuples = list(zip(times, values))
  gradients = []
  for i in range(len(time_tuples)-1):
    gradient = (time_tuples[i+1][1] - time_tuples[i][1]) / (time_tuples[i+1][0] - time_tuples[i][0])
    gradients.append(gradient)
  max_length = 0
  start_index = 0
  end_index = 0
  previous_gradient = gradients[0]
  for i in range(len(time_tuples)-1):
    length = 0
    for j in range(i, len(time_tuples)-1):
      average_gradient = (time_tuples[j+1][1] - time_tuples[i][1]) / (time_tuples[j+1][0] - time_tuples[i][0])
      percentage_change = (gradients[j] - average_gradient) / average_gradient
      if abs(percentage_change) < tolerance and average_gradient < 0:
        length += 1
        previous_gradient = average_gradient
      else:
        break
    if length > max_length:
      max_length = length
      start_index = i
      end_index = i+length
  times1, values1 = zip(*time_tuples[start_index:end_index+1])
  average_gradient = (values1[-1] - values1[0]) / (times1[-1] - times1[0])
  return average_gradient, start_index, end_index


# Setting the tolerances for all calcium_r[i] to a default 0.3 - seems to work in the majority of cases
for i in range(1, 51):
    if f'calcium_r{i}' in globals():
        exec(f'tolerance{i} = 0.3')

In [ ]:
# Running the function on all calcium_r[i] to find the line of interest in each 
for i in range(1, 51):
  try:
    calcium_r_values = globals()["calcium_r"+str(i)]  # Get the calcium_r values for the current iteration
    tolerance = globals()["tolerance"+str(i)]  # Get the tolerance value for the current iteration
    average_gradient, start_index, end_index = calculate_average_gradient(time, calcium_r_values, tolerance)
    globals()["average_gradient"+str(i)] = average_gradient
    globals()["start_index"+str(i)] = start_index
    globals()["end_index"+str(i)] = end_index
  except KeyError:
    # Skip the iteration if the variable is not defined
    continue

# Extracting the start and end index for the lines - this will be useful in plotting them later! 
for i in range(1, 51):
  try:
    start_index = globals()["start_index"+str(i)]  # Get the start index for the current iteration
    end_index = globals()["end_index"+str(i)]  # Get the end index for the current iteration
    time_tuples_values = globals()["time_tuples"+str(i)]  # Get the time_tuples values for the current iteration
    times, values = zip(*time_tuples_values[start_index:end_index+1])
    globals()["times"+str(i)] = times
    globals()["values"+str(i)] = values
  except KeyError:
    # Skip the iteration if the variable is not defined
    continue

## Plotting all of our lines on the data

In [ ]:
# Setting the number of rows and columns for the subplots
nrows = math.ceil(50 / 2)
ncols = 2

# Creating the figure and subplots
fig, axs = plt.subplots(nrows, ncols, figsize=(10, nrows*5))

# Flattening the subplot array to make it easier to iterate over
axs = axs.flatten()

# Looping through the variables to plot each one
for i, ax in enumerate(axs):
    calcium_r = "calcium_r" + str(i+1)
    values = "values" + str(i+1)
    times = "times" + str(i+1)
    if calcium_r in locals() and values in locals() and times in locals():
        ax.plot(time, locals()[calcium_r], 'b.')
        ax.plot(locals()[times], locals()[values], 'r-')
        ax.set_title("R{} calcium over time + fitted line (red)".format(i+1))
        ax.set_ylabel("Calcium level".format(i+1))
        ax.set_xlabel("Time (sec)")

## Making any reqired changes

### Refining tolerances

Once you've visualised the lines on the plots it's possible to edit the tolerance for each plot to try and improve the fit - this should only be required for a limited number of the plots as the default value works quite well! Increasing the tolerance should allow the line to get larger and work it's way through messier data, if the line is overstretched across the lag region or where the line begins to plateau it may be suitable o decrease the tolerance to improve the fit of the line.

Change the tolerance levels of specific calcium_r[i] by using the following format below:

(Do this as many times as required.)

In [ ]:
#tolerance[i] = 0.4

# *As an example - above the line fitted to calcium_r13 seems to include the lag period which is visibly
        # skewing the gradient.*

tolerance13 = 0.2

Now go back up and run the cell titled '# Running the function...'

Run that and the next cell to re-plot the data and check your fit.

Either make further adjustments or continue on to calculate the overall average gradient for this data sheet.
   
*The calcium_r13 line is looking better! Time to continue... (no need to use our last resort!)*

### Last resort

As a last resort, if a line cannot be fitted to the graph it is possible to remove that cell by:

Removing the # below and replace [i] by the calcium_r[i] of interest.
        
This may reduce the acuracy of the overall average gradient for this cell type calculated below.

In [ ]:
#del average_gradient[i]

# *No need to do this as all our lines are plotted quite nicely!*

## Calculating the overall average gradient

In [ ]:
average_decline_gradients = {} # Creating a dictionary for our average_decline_gradients for each calcium_r[i]

# Looping through the variables and adding their average decline gradients to the dictionary
for i in range(1, 51):
    gradient = "average_gradient" + str(i)
    if gradient not in locals():
        continue
    average_decline_gradients[i] = locals()[gradient]


# Defining a function to take an average of all the numbers in a dictionary.   
def average_gradient(dictionary):
    total_sum = 0
    num_elements = 0
    for key, value in dictionary.items():
        total_sum += value
        num_elements += 1
    return total_sum / num_elements, num_elements

# Using this function to calculate the overall average gradient from the average_decline_gradients dictionary.
overall_average_gradient, num_elements = average_gradient(average_decline_gradients)

print("The overall average gradient of calcium recovery of these", num_elements, "cells is:", overall_average_gradient)

# Success!
### The avereage gradient can now be compared with other disease-states and analysed further!
*Thank you for looking through my code!*